In [33]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel, Field

In [6]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [20]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", format="json", temperature=0)

In [21]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# 로컬 DB 불러오기
MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore1 = FAISS.load_local(MY_FAISS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever1 = vectorstore1.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출
MY_PDF_INDEX = "MY_PDF_INDEX"
vectorstore2 = FAISS.load_local(MY_PDF_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출

In [56]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.pydantic_v1 import BaseModel, Field

retriever_tool1 = create_retriever_tool(
    retriever1,
    name="web_search",
    description="엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다",
)

retriever_tool2 = create_retriever_tool(
    retriever2,
    name="pdf_search",
    description="생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다",
)

tools = [retriever_tool1, retriever_tool2]
tools

# class Add(BaseModel): 
#     a: int = Field(..., description="First integer")
#     b: int = Field(..., description="Second integer")
    
# class Multiply(BaseModel): 
#     a: int = Field(..., description="First integer")
#     b: int = Field(..., description="Second integer")
    
# tools = [Add, Multiply]

"[Tool(name='web_search', description='엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x1487dd6c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x317e52ea0>, search_kwargs={'k': 5}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\\n\\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x1487ddbc0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x317e52ea0>, search_kwargs={'k': 5}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\\n\\n')), Tool(name='pdf_search', description='생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다', args_schema=<class 'langchain_core

In [44]:
# llm_with_tools = llm.bind_tools(tools=tools)
# response = llm_with_tools.invoke("What is 3*12=? Also, What is 11+43?")
# print(f"response: {response}")
# print(f"tool_calls: {response.tool_calls}")

In [63]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
question에 알맞는 답을 하기 위해 어떤 tool을 사용하면 좋을지 tool 하나를 선택해줘

<tools>
{tools}
</tools>

<question>
{question}
</question>

# answer :
"""
    ),
])

chain = (
    {"tools": "str(tools)", "question": "RunnablePassthrough()"}
    | prompt 
    # | llm
    )

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>